# Module 4: Introduction to Semantic Kernel Processes

## Theory Module

### Prerequisites

- Python 3.8 or later
- Basic understanding of Python programming
- OpenAI API key or Azure OpenAI access
- Semantic Kernel library installed
- Understanding of previous modules (Skills, Memory, Planning)

```python
# Setup - call this once at the beginning of the notebook to add the parent directory to the path
import os
import sys
import asyncio
from enum import Enum
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)
sys.path.append(grandparent_dir)
```

### 1. What are Semantic Kernel Processes?

Semantic Kernel Processes provide a framework for creating complex, stateful workflows that can:
- Handle multiple events and states
- Manage sequential and parallel operations
- Maintain state across executions
- Respond to events conditionally
- Create reusable process components

Think of processes as a way to create sophisticated AI workflows that can maintain their state and handle complex event flows, similar to a state machine but with AI capabilities built in.

### 2. Key Concepts

#### 2.1 Process Steps
Steps are the building blocks of processes. Each step can:
- Have its own state
- Handle events
- Emit events
- Execute functions

Here's a simple example of a step:

```python
from semantic_kernel.processes.kernel_process.kernel_process_step import KernelProcessStep
from semantic_kernel.functions.kernel_function_decorator import kernel_function

class IntroStep(KernelProcessStep):
    @kernel_function
    async def print_intro_message(self):
        print("Welcome to Processes in Semantic Kernel.\n")
```

#### 2.2 Process Events
Events are how steps communicate with each other. They can be:
- Input events that start processes
- Internal events between steps
- Output events that signal completion

```python
class CommonEvents(Enum):
    StartProcess = "startProcess"
    ProcessComplete = "processComplete"
    ErrorOccurred = "errorOccurred"
```

#### 2.3 Process State
Steps can maintain state across executions:

```python
from semantic_kernel.kernel_pydantic import KernelBaseModel

class ProcessState(KernelBaseModel):
    counter: int = 0
    last_event: str = ""
    
class StatefulStep(KernelProcessStep[ProcessState]):
    state: ProcessState
    
    async def activate(self, state: KernelProcessStepState[ProcessState]):
        self.state = state.state or ProcessState()
```

### 3. Building a Process

Let's create a simple restaurant order process to demonstrate these concepts:

```python
from semantic_kernel.processes.process_builder import ProcessBuilder

class OrderEvents(Enum):
    OrderReceived = "orderReceived"
    OrderValidated = "orderValidated"
    PaymentProcessed = "paymentProcessed"
    OrderPrepared = "orderPrepared"
    OrderComplete = "orderComplete"

class ValidateOrderStep(KernelProcessStep):
    @kernel_function
    async def validate_order(self, context):
        print("Validating order...")
        await context.emit_event(OrderEvents.OrderValidated)

class ProcessPaymentStep(KernelProcessStep):
    @kernel_function
    async def process_payment(self, context):
        print("Processing payment...")
        await context.emit_event(OrderEvents.PaymentProcessed)

class PrepareOrderStep(KernelProcessStep):
    @kernel_function
    async def prepare_order(self, context):
        print("Preparing order...")
        await context.emit_event(OrderEvents.OrderPrepared)

# Build the process
process = ProcessBuilder(name="RestaurantOrderProcess")

# Add steps
validate_step = process.add_step(ValidateOrderStep)
payment_step = process.add_step(ProcessPaymentStep)
prepare_step = process.add_step(PrepareOrderStep)

# Define the flow
process.on_input_event(OrderEvents.OrderReceived).send_event_to(validate_step)
validate_step.on_event(OrderEvents.OrderValidated).send_event_to(payment_step)
payment_step.on_event(OrderEvents.PaymentProcessed).send_event_to(prepare_step)
prepare_step.on_event(OrderEvents.OrderPrepared).send_event_to(OrderEvents.OrderComplete)
```

### 4. Advanced Process Features

#### 4.1 Parallel Processing
Processes can handle multiple events simultaneously:

```python
class FishAndChipsProcess:
    @staticmethod
    def create_process():
        process = ProcessBuilder("FishAndChipsProcess")
        
        fry_fish_step = process.add_step(FryFishStep)
        make_chips_step = process.add_step(MakeChipsStep)
        plate_step = process.add_step(PlateStep)
        
        # Start both steps in parallel
        process.on_input_event(Events.StartCooking).send_event_to([
            fry_fish_step,
            make_chips_step
        ])
        
        # Wait for both to complete before plating
        fry_fish_step.on_event(Events.FishReady).send_event_to(plate_step)
        make_chips_step.on_event(Events.ChipsReady).send_event_to(plate_step)
```

#### 4.2 Stateful Components
Here's an example of a stateful knife-sharpening process:

```python
class KnifeState(KernelBaseModel):
    sharpness: int = 100
    uses_since_sharpening: int = 0

class CuttingStep(KernelProcessStep[KnifeState]):
    @kernel_function
    async def cut_ingredient(self, context):
        if self.state.sharpness < 20:
            await context.emit_event(Events.NeedSharpening)
            return
            
        self.state.uses_since_sharpening += 1
        self.state.sharpness -= 5
        await context.emit_event(Events.CuttingComplete)

class SharpeningStep(KernelProcessStep[KnifeState]):
    @kernel_function
    async def sharpen_knife(self, context):
        self.state.sharpness = 100
        self.state.uses_since_sharpening = 0
        await context.emit_event(Events.SharpeningComplete)
```

### 5. Best Practices

1. **Event Naming**
   - Use clear, descriptive event names
   - Follow a consistent naming convention
   - Group related events in Enums

2. **State Management**
   - Keep state minimal and focused
   - Use proper typing for state objects
   - Consider state persistence needs

3. **Error Handling**
   - Define error events
   - Include recovery paths
   - Log state changes

4. **Process Design**
   - Break complex processes into subprocesses
   - Use clear step boundaries
   - Consider reusability

### 6. Running a Process

```python
async def run_process():
    kernel = Kernel()
    kernel.add_service(OpenAIChatCompletion(service_id="default"))
    
    # Build process
    process = create_restaurant_process()
    
    # Start process
    await start(
        process=process,
        kernel=kernel,
        initial_event=OrderEvents.OrderReceived
    )

# Run the process
if __name__ == "__main__":
    asyncio.run(run_process())
```

### 7. Common Use Cases

1. **Business Workflows**
   - Order processing
   - Document approval
   - Customer service flows

2. **AI Interactions**
   - Multi-turn conversations
   - Complex reasoning tasks
   - Decision trees

3. **Data Processing**
   - ETL workflows
   - Data validation chains
   - Analysis pipelines

4. **System Automation**
   - Deployment processes
   - Monitoring workflows
   - Maintenance tasks

### Resources

For more information, check:
- [Semantic Kernel Process Documentation](https://learn.microsoft.com/semantic-kernel/processes/)
- [Process Framework Examples](https://github.com/microsoft/semantic-kernel/samples)
- [Best Practices Guide](https://learn.microsoft.com/semantic-kernel/processes/best-practices)